<a href="https://colab.research.google.com/github/Jonathangadeaharder/SubtitleMaker/blob/main/Subtitle_Ultimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python3.10-venv
!pip list
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!source /content/gdrive/MyDrive/colab_env/bin/activate

In [ ]:
!pip uninstall -y lida flask dask distributed fiona pip-tools tensorflow-datasets tensorflow-metadata llmx pandas-gbq typer torchdata torchtext torchvision> /dev/null 2>&1
!pip install moviepy openai-whisper pydub ffmpeg-python pytube sacremoses sentencepiece transformers librosa
!pip install flash-attn --no-build-isolation accelerate sacremoses sentencepiece torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_c

In [ ]:
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



In [ ]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
import warnings
from pytube import YouTube
import whisper
import torch
import sentencepiece

In [ ]:
video_link="Drive"
language="de"
double=False
single_original=False
dailymotion=False
fast=False

In [ ]:
from pytube import YouTube
import os
import subprocess # Import subprocess for running external commands
from google.colab import drive
drive.mount('/content/gdrive')
if "youtube" in video_link or "youtu.be" in video_link:
    print("Youtube Option Selected")
    directory = '.'
    # Create a YouTube object
    yt = YouTube(video_link)
    # Try to select the 480p stream
    try:
        stream = yt.streams.filter(progressive=True, file_extension='mp4', resolution='480p').first()
        # If 480p stream is not available, select 360p
        if stream is None:
            stream = yt.streams.filter(progressive=True, file_extension='mp4', resolution='360p').first()
            if stream is None:
                print("Suitable video resolution not found.")
    except Exception as e:
        print("YouTube video not found or other error:", e)
        raise RuntimeError
    video_file = stream.download()
else:
    if "Drive" in video_link:
        print("Drive Option Selected")
        # Assuming this path is correct, modify it according to your environment
        directory = '/content/gdrive/MyDrive'
    elif video_link:
        print("Download Option Selected")
        try:
            subprocess.run(["wget", video_link], check=True)
        except Exception as e:
            print("Error downloading file:", e)
        directory = '.'
    else:
        directory = '.'

    files = os.listdir(directory)
    # Filter for mp4 files
    import os

# Get list of MP4 files
mp4_files = [file for file in os.listdir(directory) if file.endswith('.mp4') and os.path.exists(os.path.join(directory, file))]
# Find file with latest creation time
latest_mp4 = max(mp4_files, key=lambda x: os.path.getctime(os.path.join(directory, x)))

if latest_mp4:
    print(f"Latest MP4 file: {latest_mp4}")
else:
    print("No MP4 file found")
video_file=os.path.join(directory, latest_mp4)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive Option Selected
Latest MP4 file: Sex.and.the.City.S05E07.German.720p.WebHD.h264-FKKTV.mkv sub español.mp4


In [ ]:
if(fast):
  model = whisper.load_model("base",device='cuda',download_root="cache",in_memory=False)
else:
  model = whisper.load_model("large-v3",device='cuda',download_root="cache",in_memory=False)
import shutil
import gc
shutil.rmtree("cache")
gc.collect()

100%|██████████████████████████████████████| 2.88G/2.88G [00:27<00:00, 111MiB/s]


9

In [ ]:
import time
from moviepy.editor import VideoFileClip
# Extract audio from video
print("Starting audio extraction...")
with VideoFileClip(video_file) as video:
    audio = video.audio
    audio_file = "extracted_audio.wav"
    audio.write_audiofile(audio_file)
print("Audio extracted successfully.")

Starting audio extraction...
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Audio extracted successfully.


In [ ]:
audio = whisper.load_audio("extracted_audio.wav")

In [ ]:
res = model.transcribe(audio,language=language,verbose=False,condition_on_previous_text=False)

 98%|█████████▊| 176229/179229 [06:32<00:06, 448.55frames/s] 


In [ ]:
sentences = res['segments'] # Ihre Wort-Zeitstempel-Daten

In [ ]:
def format_time(timestamp):
    """ Convert timestamp to SRT time format """
    hours, remainder = divmod(timestamp, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = int((seconds % 1) * 1000)
    return "{:02}:{:02}:{:02},{:03}".format(int(hours), int(minutes), int(seconds), milliseconds)

def estimate_end_time(start_time, text, words_per_minute=100):
    """ Estimate the end time based on text length and speaking speed """
    word_count = len(text.split())
    duration_in_seconds = (word_count / words_per_minute) * 60
    return start_time + duration_in_seconds

def dict_to_srt(subtitles):
    """ Convert dictionary to SRT format with line breaks """
    srt_string = ""
    crap=0
    for index, subtitle in enumerate(subtitles):
        start_time = subtitle["start"]
        if not subtitle["end"]:
            crap+=1
            if(index<len(subtitles)-1 and subtitles[index+1]["start"]):
                end_time = subtitles[index+1]["start"]
            else:
                end_time = estimate_end_time(start_time, subtitle["text"])
        else:
            end_time=subtitle["end"]
        formatted_start_time = format_time(start_time)
        formatted_end_time = format_time(end_time)
        srt_string += f"{index + 1}\n{formatted_start_time} --> {formatted_end_time}\n"+subtitle["text"]+"\n\n"
    print(f"craps:{crap}")
    return srt_string

# Convert to SRT
srt_output = dict_to_srt(sentences)
srt_filename = "/content/gdrive/My Drive/output.srt"
with open(srt_filename, "w", encoding="utf-8") as file:
    file.write(srt_output)


craps:0


In [ ]:
import gc
del model
del res
del srt_output
gc.collect()

0

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm.notebook import tqdm
import re
if(fast):
  model_name="Helsinki-NLP/opus-mt-tc-de-es"
else:
  model_name="Helsinki-NLP/opus-mt-tc-big-de-es"
model = MarianMTModel.from_pretrained(model_name).to('cuda')
tokenizer = MarianTokenizer.from_pretrained(model_name)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/823k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/823k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
def insert_line_breaks(text, max_line_length=50):
    """ Insert line breaks in text to ensure each line is within the max_line_length """
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        if len(current_line) + len(word) + 1 > max_line_length:
            lines.append(current_line)
            current_line = word
        else:
            if current_line:
                current_line += " "
            current_line += word

    lines.append(current_line)  # Add the last line
    return "\n".join(lines)

def translate_text(text, model, tokenizer):
    """Translate text using a MarianMT model."""
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
def read_srt_file(input_file):
    """Read an SRT file and return its content."""
    encodings = ['utf-8', 'ISO-8859-1', 'cp1252']
    for encoding in encodings:
        try:
            with open(input_file, "r", encoding=encoding) as file:
                return file.readlines()
        except UnicodeDecodeError:
            continue
    raise ValueError("None of the encodings worked. File might be corrupted or in an unknown encoding.")

def translate_srt_file(input_file, output_file):
    """Translate an SRT file."""
    lines = read_srt_file(input_file)
    with open(output_file, "w", encoding="utf-8") as file:
        for line in tqdm(lines, desc="Processing Subtitles"):
            if re.match(r'^\d+$', line) or re.match(r'^\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}$', line):
                file.write(line)
            elif line.strip():
                # First translate from German
                stripped_line=line.strip()
                if(double): file.write('<font size="22px" color="yellow">'+insert_line_breaks(stripped_line)+"</font>\n")
                translated_line=translate_text(stripped_line,model,tokenizer)
                if(double): file.write('<font size="10px">'+insert_line_breaks(translated_line) + "</font>\n")
                else:
                    if(single_original):
                        file.write('<font size="22px">'+insert_line_breaks(stripped_line)+ "</font>\n")
                    else:
                        file.write('<font size="22px">'+insert_line_breaks(translated_line) + "</font>\n")
            else:
                file.write('\n')

# Example usage
input_srt_file = '/content/gdrive/My Drive/output.srt'
output_srt_file = "/content/gdrive/My Drive/translated_output.srt"

translate_srt_file(input_srt_file, output_srt_file)

print(f"Translated SRT file saved as: {output_srt_file}")

Processing Subtitles:   0%|          | 0/1888 [00:00<?, ?it/s]

Translated SRT file saved as: /content/gdrive/My Drive/translated_output.srt


In [ ]:
import os
import subprocess

def apply_srt_subtitles_to_video(video_path, srt_path, output_path):
    cmd = [
        "ffmpeg",
        "-y",
        "-hwaccel_device", "0",  # Specify GPU device index (usually 0)
        "-i", video_path,
        "-vf", "subtitles="+ srt_path,
        "-c:a", "copy",
        "-c:v", "h264_nvenc",  # GPU-accelerated codec
        output_path
    ]
    with open('ffmpeg_log.txt', 'w') as log_file:
        subprocess.run(cmd, check=True, stderr=log_file)

file_name_without_extension = os.path.splitext(os.path.basename(video_file))[0]
# Paths
output_video_path = "/content/gdrive/My Drive/"+file_name_without_extension + " sub español.mp4"
# Apply SRT subtitles to the video
apply_srt_subtitles_to_video(video_file, "/content/gdrive/My Drive/translated_output.srt", output_video_path)
# Optionally, remove the intermediate file
os.remove(video_file)

In [ ]:
del model
gc.collect()

19

In [ ]:
import requests

url = "https://api.dailymotion.com/oauth/token"
payload = {
    "grant_type": "password",
    "client_id": "ce7016bf02819231dbc8",
    "client_secret": "7baef2614adef0b9480530cf20fbb41f01ca7751",
    "username": "jonathangadeah@icloud.com",
    "password": "mF8$NL4f@p8$Xa",
    "scope": "manage_videos",
}
response = requests.post(url, data=payload)
response_data = response.json()
access_token = response_data['access_token']

In [ ]:
def upload_video(access_token, video_path):
    # Step 1: Create an upload URL
    url = "https://api.dailymotion.com/file/upload"
    headers = {"Authorization": f"Bearer {access_token}"}
    upload_url_response = requests.get(url, headers=headers)
    upload_url = upload_url_response.json()['upload_url']

    # Step 2: Upload the file
    files = {'file': open(video_path, 'rb')}
    upload_response = requests.post(upload_url, files=files)
    file_url = upload_response.json()['url']

    # Step 3: Create the video on Dailymotion
    create_video_url = "https://api.dailymotion.com/me/videos"
    data = {
        "url": file_url,
        "title": file_name_without_extension + " sub español",
        "channel": "news",
        "published": "true",
        "is_created_for_kids":"false",
        "language": "de",
        "private":"true",

    }
    create_video_response = requests.post(create_video_url, data=data, headers=headers)
    return create_video_response.json()

if(dailymotion):
    uploaded_video = upload_video(access_token, output_video_path)
    print(uploaded_video)
